In [1]:
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0)

In [2]:
import os
# --- Uncomment to use only CPU (e.g. GPU memory is too small)
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

# import sys
# sys.path.append("/usr/local/cuda-10.1/bin")
# os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-10.1/lib64"

import tensorflow as tf
tf.test.is_gpu_available(cuda_only=True) 

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
# reate logger - nocer formatting
import logging

logger = logging.getLogger()
logging._warn_preinit_stderr = 0
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s: %(message)s')
ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.handlers = [ch]
logger.info("Logger started.")

2020-12-04 11:24:00,103: Logger started.


In [4]:
import sqlite3 
import numpy as np

def adapt_array(arr):
    """
    Save Numpy array to SqLite.
    Source:
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    """ 
    Load Numpy array from Sqlite.
    Source:
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)
data_path = os.path.join(os.getcwd(), "samplesGeneration", "prepared_data.bd")
source = sqlite3.connect(data_path, detect_types=sqlite3.PARSE_DECLTYPES, check_same_thread=False)
db = sqlite3.connect(':memory:')
source.backup(db)

In [5]:
import os
# --- Uncomment to use only CPU (e.g. GPU memory is too small)
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

# import sys
# sys.path.append("/usr/local/cuda-10.1/bin")
# os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-10.1/lib64"

import tensorflow as tf
tf.test.is_gpu_available(cuda_only=True) 

True

In [10]:
import io
sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)

db_cursor = db.cursor()
sql_query = "SELECT COALESCE(MAX(id)+1, 0) FROM data"
db_cursor.execute(sql_query)
number_of_samples = db_cursor.fetchone()[0]
print("number of samples: ", number_of_samples)

sql_query = "SELECT x, y FROM data WHERE id == 1"
db_cursor.execute(sql_query)

number of samples:  1703921


In [7]:
import tensorflow.keras as keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras from SQLite database'
    def __init__(self, path, indexes=None, batch_size=25,
                 shuffle=True, dim=(117, 1), dim_y=(114, 1), column="voxels"):
        'Initialization'
        if indexes is None:
            raise Exception("Indexes need to be provided!")
        sqlite3.register_adapter(np.ndarray, adapt_array)
        sqlite3.register_converter("array", convert_array)
        self.path = path
        self.dim = dim
        self.dim_y = dim_y
        self.db = sqlite3.connect(self.path, detect_types=sqlite3.PARSE_DECLTYPES, check_same_thread=False)
        self.db_cursor = self.db.cursor()
        self.N = len(indexes) - 1
        self.sample_index = indexes
        self.column = column
        self.batch_size = int(batch_size)
        self.shuffle = shuffle
        self.on_epoch_end()
        self.db.close()


    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.N / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data - generates indexes of the batch'
        # Generate indexes of the batch
        samples_batch = np.arange((index) * self.batch_size, (index+1) * self.batch_size)

        # Generate data
        while True:
            try:
                x, y = self.__data_generation(samples_batch)
                break
            except ValueError:
                continue

        return  x, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle:
            np.random.shuffle(self.sample_index)

    def __data_generation(self, samples_batch):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        x = np.empty((self.batch_size, *self.dim))
        y = np.empty(shape=(self.batch_size, *self.dim_y))
        inds = self.sample_index[samples_batch]
        sqlite3.register_adapter(np.ndarray, self.adapt_array)
        sqlite3.register_converter("array", self.convert_array)
        db = sqlite3.connect(self.path, detect_types=sqlite3.PARSE_DECLTYPES)
        db_cursor = db.cursor()
        sql_query = "SELECT x, y FROM data WHERE id in ({index})".\
            format(column=self.column, index=','.join(str(ind) for ind in inds))
        db_cursor.execute(sql_query)
        try:
            for i, line in zip(range(self.batch_size), db_cursor.fetchall()):
                if line is None:
                    x[i, :] = x[i - 1, :]
                    y[i] = y[i - 1]
                    continue # Bad, temporary solution
                x[i, :] = line[0].reshape(self.dim)
                y[i] = line[1].reshape(self.dim_y)
        except TypeError as e:
            print(sql_query)
            raise e
        # x.reshape(self.batch_size, *self.dim, 1)
        y = y.reshape(self.batch_size, *self.dim_y)
        return x, y
    
    def adapt_array(self, arr):
        """
        Save Numpy array to SqLite.
        Source:
        http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
        """
        out = io.BytesIO()
        np.save(out, arr)
        out.seek(0)
        return sqlite3.Binary(out.read())

    def convert_array(self, text):
        """ 
        Load Numpy array from Sqlite.
        Source:
        http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
        """
        out = io.BytesIO(text)
        out.seek(0)
        return np.load(out)
    
test_train_ratio = 0.1

test_size = int(test_train_ratio * number_of_samples)

indexes = np.arange(0, number_of_samples - 1)
np.random.shuffle(indexes)
train_indexes, val_indexes = indexes[:-test_size], indexes[test_size:]

training_generator = DataGenerator(data_path, indexes=train_indexes, batch_size=32, dim=(117, 1), dim_y=(36, 1))
validation_generator = DataGenerator(data_path, indexes=val_indexes, batch_size=32, dim=(117, 1), dim_y=(36, 1))

In [8]:
import time
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

best_model_file = "model.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_loss', mode='min',verbose=1, save_best_only=True)

models_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs")

In [9]:
# Try learning for all of verts:

verts = range(14)
pivot_vert = 3

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import LSTM
from collections import deque
from tensorflow.keras.callbacks import EarlyStopping
import time


models_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs", "model_{}".format(time.time()))


def create_and_train_network(generator, val_generator, epohs=10):
    log_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs", "model_{}".format(time.time()))
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    def create_model():
        model = Sequential()
        #model.add(LSTM(512, input_shape=generator.dim, return_sequences=True))
        #model.add(Dropout(0.2))
        model.add(LSTM(128, input_shape=generator.dim, return_sequences=False))
        model.add(Dropout(0.4))
        #model.add(Flatten())
        model.add(Dense(np.prod(generator.dim_y), activation='linear'))
        model.add(Reshape((np.prod(generator.dim_y), 1)))
        model.compile(loss='mae', optimizer='adam')
        return model

    logger.info("Create model...")
    model = create_model()
    model.summary()
    logger.info("Fit model..")
    network = model.fit_generator(training_generator, verbose=1, validation_data=validation_generator, 
                                  epochs=epohs, callbacks=[early_stopping, best_model, tensorboard_callback])
    logger.info("Summary")
    model.summary()
    model.save(os.path.join(models_dir, "model_all_points_{}".format(sample_length)))
    return network


logger.info("start")
network = create_and_train_network(training_generator, validation_generator)

2020-12-04 11:24:06,143: start
2020-12-04 11:24:06,143: Create model...
2020-12-04 11:24:06,768: Fit model..


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               66560     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 36)                4644      
_________________________________________________________________
reshape (Reshape)            (None, 36, 1)             0         
Total params: 71,204
Trainable params: 71,204
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Please use Model.fit, which supports generators.


2020-12-04 11:24:06,768: From <ipython-input-9-20e079fc1eb0>:32: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.


  ...
    to  
  ['...']


2020-12-04 11:24:06,789: sample_weight modes were coerced from
  ...
    to  
  ['...']


...

2020-12-04 11:24:06,831: sample_weight modes were coerced from
  ...
    to  
  ['...']


Train for 47922 steps, validate for 47922 steps
Epoch 1/100
47920/47922 [============================>.] - ETA: 0s - loss: 0.1811
Epoch 00001: val_loss improved from inf to 0.07187, saving model to model.h5
47922/47922 [==============================] - 1771s 37ms/step - loss: 0.1811 - val_loss: 0.0719
Epoch 2/100
47920/47922 [============================>.] - ETA: 0s - loss: 0.1027
Epoch 00002: val_loss improved from 0.07187 to 0.04651, saving model to model.h5
47922/47922 [==============================] - 1767s 37ms/step - loss: 0.1027 - val_loss: 0.0465
Epoch 3/100
47920/47922 [============================>.] - ETA: 0s - loss: 0.0955
Epoch 00003: val_loss improved from 0.04651 to 0.04056, saving model to model.h5
47922/47922 [==============================] - 1759s 37ms/step - loss: 0.0955 - val_loss: 0.0406
Epoch 4/100
47919/47922 [============================>.] - ETA: 0s - loss: 0.0933
Epoch 00004: val_loss did not improve from 0.04056
47922/47922 [==============================

2020-12-04 16:43:56,564: Summary


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               66560     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 36)                4644      
_________________________________________________________________
reshape (Reshape)            (None, 36, 1)             0         
Total params: 71,204
Trainable params: 71,204
Non-trainable params: 0
_________________________________________________________________


NameError: name 'models_dir' is not defined

In [27]:
from keras import backend as K

model = tf.keras.models.load_model(best_model_file)

tf.saved_model.save(model, "model\\saved_model")

INFO:tensorflow:Assets written to: model\saved_model\assets


2020-12-04 17:58:57,257: Assets written to: model\saved_model\assets


In [28]:
saver = tf.train.Saver()

AttributeError: module 'tensorflow_core._api.v2.train' has no attribute 'Saver'